In [1]:
import numpy as np
import mdtraj as md
import matplotlib.pyplot as plt
import nglview as nv

import openmm as mm
import openmm.app as app
import openmm.unit as unit
from mdtraj.reporters import HDF5Reporter
import copy
import nglview as nv
import random

# # Path: pymdna/__init__.py, prototype of the package and atomic is not properly referenced in the package at genertors.py now I just explicitly define the path loction
# import pymdna as mdna
import sys
sys.path.append('/Users/thor/surfdrive/Projects/pymdna/')
import pymdna as mdna 
%load_ext autoreload
%autoreload 2

joblib is not installed. Falling back to sequential computation.


/Users/thor/surfdrive/Projects/PMCpy/pmcpy/Evals/PyLk/pylk/writhemap.py:16: UserWarning: Cython version of writhemap (PyLk) not compiled. Defaulting to numba implementation. Consider compiling the cython version.
  warnings.warn(
/Users/thor/surfdrive/Projects/PMCpy/pmcpy/Evals/PyLk/pylk/eval_link.py:10: UserWarning: Cython version of linkingnumber (PyLk) not compiled. Defaulting to numba implementation. Consider compiling the cython version.
  warnings.warn(


In [19]:
from Bio.SVDSuperimposer import SVDSuperimposer

def get_base_vectors(res):
       """Compute base vectors from reference base."""
       ref_base = mdna.ReferenceBase(res)
       return np.array([ref_base.b_R, ref_base.b_L, ref_base.b_D, ref_base.b_N]).swapaxes(0,1)

def get_rot_mat_trans(x,y):
    # load super imposer
    sup = SVDSuperimposer()

    # Set the coords, y will be rotated and translated on x
    sup.set(x, y)

    # Do the leastsquared fit
    sup.run()

    # Get the rms
    rms = sup.get_rms()

    # Get rotation (right multiplying!) and the translation
    rot, tran = sup.get_rotran()
    return rot, tran

# Function to calculate positions from origin and vectors
def calculate_positions(triad):
    
    origin = triad[0]
    vectors = triad[1:]
    # Each row in vectors is added to the origin to get the end position
    end_positions = origin + vectors
    # Combine the origin with these end positions
    positions = np.vstack([origin, end_positions])
    return positions


def align_to_ref(traj, ref =  np.array([[0,0,0.0],[1,0,0],[0,1,0],[0,0,1]])):
    vectors = get_base_vectors(traj)
    positions = calculate_positions(vectors[0])
    ref_position =  calculate_positions(ref)
    rot, tran = get_rot_mat_trans(ref_position,positions)
    new_xyz = np.dot(traj.xyz[0], rot) + tran
    traj.xyz[0] = new_xyz
    return traj

In [16]:
dna = mdna.make('GCGCG')
dna.methylate(CpG=True)
traj = dna.get_traj()
meth = traj.atom_slice(traj.top.select('resid 1'))
meth = align_to_ref(meth)
meth.save('BDNA_CM.pdb')
meth.save('BDNA_CM.h5')
view = nv.show_mdtraj(meth)
view.clear()
view.add_ball_and_stick()
view


Start rescaling spline based on requested number of base pairs.
	This requires recomputation of the control points to match the desired number of base pairs.
	Spline scaled to match the target number of base pairs: 5

Methylate all C in CpG context, superseeds methylations list.
Methtylating: [1, 3]


NGLWidget()

In [48]:
dna = mdna.make('GGG')
dna.methylate(methylations=[1])
traj = dna.get_traj()
meth = traj.atom_slice(traj.top.select('resid 1'))
meth = align_to_ref(meth)
meth.save('BDNA_GM.pdb')
meth.save('BDNA_GM.h5')
view = nv.show_mdtraj(meth)
view.clear()
view.add_ball_and_stick()
view


Start rescaling spline based on requested number of base pairs.
	This requires recomputation of the control points to match the desired number of base pairs.
	Spline scaled to match the target number of base pairs: 3



NGLWidget()

In [45]:
dna = mdna.make('GAG')
dna.flip(fliplist=[1])
traj = dna.get_traj()
hoog = traj.atom_slice(traj.top.select('resid 1'))

AAAA =  mdna.make('AAAA').get_traj()

ref = AAAA.atom_slice(AAAA.top.select('resid 1'))
ref = align_to_ref(ref)
ref_atoms = ref.top.select('name OP1 OP2 P')
hoog_atoms = hoog.top.select('name OP1 OP2 P')
print(ref_atoms)
print(hoog_atoms)   
hoog.superpose(ref,atom_indices=hoog_atoms, ref_atom_indices=ref_atoms)
hoog.save('BDNA_AH.pdb')
hoog.save('BDNA_AH.h5')


Start rescaling spline based on requested number of base pairs.
	This requires recomputation of the control points to match the desired number of base pairs.
	Spline scaled to match the target number of base pairs: 3

Flipped residues [1] by 3.141592653589793 radians

Start rescaling spline based on requested number of base pairs.
	This requires recomputation of the control points to match the desired number of base pairs.
	Spline scaled to match the target number of base pairs: 4

[0 1 2]
[0 1 2]


In [44]:
view = nv.show_mdtraj(hoog.stack(ref))  
view.clear()
view.add_ball_and_stick()
view

NGLWidget()

In [27]:

hoog = align_to_ref(hoog, ref= np.array([[0,0,0.0],[1,0,0],[0,1,0],[0,0,1]]))
hoog.save('BDNA_AH.pdb')
#hoog.save('BDNA_AH.h5')
view = nv.show_mdtraj(hoog)
view.clear()
view.add_ball_and_stick()
view


Start rescaling spline based on requested number of base pairs.
	This requires recomputation of the control points to match the desired number of base pairs.
	Spline scaled to match the target number of base pairs: 3

Flipped residues [1] by 3.141592653589793 radians


NGLWidget()

In [51]:
base_pair_map = {'A':'T','T':'A','G':'C','C':'G','U':'A','D':'G','E':'T','L':'M','M':'L','B':'S','S':'B','Z':'P','P':'Z','CM':'G','AH':'T','GM':'C'}
        
reference_bases = {base: md.load_hdf5(mdna.utils.get_data_file_path(f'./atomic/bases/BDNA_{base}.h5')) for base in base_pair_map.keys()}

bases = list(reference_bases.values())
# order = [['A', 'T', 'G', 'C', 'U'], 
#          ['B', 'S', 'Z', 'P','CM'], 
#          ['E', 'D','L', 'M', 'AH']]

order = [['A', 'T', 'G', 'C'],
        ['AH', 'U', 'GM', 'CM'], 
         ['B', 'S', 'P', 'Z'], 
         ['E', 'D','L', 'M']]


# Initialize the trajectory with the first base
traj = reference_bases[order[0][0]]

# Spacing parameters (in angstroms, adjust as needed)
horizontal_spacing = 1.2  # Spacing between bases within a row
vertical_spacing = 1.5    # Spacing between rows

# Base positions tracking
y_position = 0  # Start at the top-most row and work downwards

for row in order:
    x_position = 0  # Reset x position for each new row
    for i, base in enumerate(row,1):
        if i == 0 and row == order[0]:
            # Already initialized with the first base
            continue
        # Move base in x and y direction
        reference_bases[base].xyz[0] = reference_bases[base].xyz[0] + np.array([-x_position, y_position, 0])
        # Stack the base to the trajectory
        traj = traj.stack(reference_bases[base])
        # Increment x position for the next base in the row
        x_position += horizontal_spacing
    # Decrement y position for the next row to position it below the current one
    y_position -= vertical_spacing

In [52]:
subtraj = traj.atom_slice(traj.top.select('not element H'))
subtraj.save_pdb('all_bases.pdb')
subtraj.save_hdf5('all_bases.h5')
view = nv.show_mdtraj(subtraj)    
view.clear()
view.add_representation('licorice', selection='all')
view

NGLWidget()